In [0]:
%pip install databricks-labs-dqx

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from databricks.labs.dqx import check_funcs
from databricks.labs.dqx.engine import DQEngine
from databricks.sdk import WorkspaceClient
from databricks.labs.dqx.rule import DQForEachColRule

# Create a sample DataFrame
data = [
    ("Alice", 1, "USA"),
    ("Bob", 2, "Canada"),
    ("Charlie", None, "Mexico"),
    (None, 4, None),
]

columns = ["Name", "ID", "Country"]
input_df = spark.createDataFrame(data, columns)

# Defining a simple data quality rule for 3 columns to check that the Country, ID and Name columns are not null.

checks = [
    *DQForEachColRule(
        criticality="error",
        check_func=check_funcs.is_not_null,
        columns=["Country", "ID", "Name"],  # Apply to multiple columns
    ).get_rules()
]

print("Original DataFrame:")
display(input_df)

# Create a DQEngine instance using the Databricks Workspace client
# This ensures DQX can interact with your Databricks environment's services.

dq_engine = DQEngine(WorkspaceClient())

# Run the data quality check
valid_df, quarantine_df = dq_engine.apply_checks_and_split(input_df, checks)

print("\nValid Rows (passed all checks):")
display(valid_df)

print("\nQuarantined Rows (failed one or more checks):")
display(quarantine_df)

Original DataFrame:


Name,ID,Country
Alice,1,USA
Bob,2,Canada
Charlie,null,Mexico
null,4,null



Valid Rows (passed all checks):


Name,ID,Country
Alice,1,USA
Bob,2,Canada



Quarantined Rows (failed one or more checks):


Name,ID,Country,_errors,_warnings
Charlie,null,Mexico,"List(List(id_is_null, Column 'ID' value is null, List(ID), null, is_not_null, 2025-07-11T19:44:35.532Z, Map()))",null
null,4,null,"List(List(country_is_null, Column 'Country' value is null, List(Country), null, is_not_null, 2025-07-11T19:44:35.532Z, Map()), List(name_is_null, Column 'Name' value is null, List(Name), null, is_not_null, 2025-07-11T19:44:35.532Z, Map()))",null
